In [ ]:
!pip install yfinance
!pip install pytickersymbols

In [ ]:
import yfinance
import os
import requests 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from tqdm.notebook import tqdm
from time import sleep
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz, plot_tree
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error, precision_score, recall_score, accuracy_score, f1_score
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC, SVR

# Ejemplo: 
Crear un árbol de decisión para estimar si una acción va a bajar o subir el día siguiente dado 2 variables: 1. el número de veces ha ido subiendo (sreak/racha) y 2. el sector al que pertenece

In [ ]:
sp500_url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500, *_=pd.read_html(sp500_url)

In [ ]:
sample = sp500[['Symbol','GICS Sector', 'GICS Sub-Industry']]
#sample = sample.tail(50).reset_index()

In [ ]:
sample#.Symbol[0]

,Symbol,GICS Sector,GICS Sub-Industry
0,MMM,Industrials,Industrial Conglomerates
1,ABT,Health Care,Health Care Equipment
2,ABBV,Health Care,Pharmaceuticals
3,ABMD,Health Care,Health Care Equipment
4,ACN,Information Technology,IT Consulting & Other Services
...,...,...,...
500,YUM,Consumer Discretionary,Restaurants
501,ZBRA,Information Technology,Electronic Equipment & Instruments
502,ZBH,Health Care,Health Care Equipment
503,ZION,Financials,Regional Banks


In [ ]:
# sample=sp500.groupby("GICS Sector")["Symbol"].apply(lambda v: v.sample(1,replace=True,random_state=314))
# sample

In [ ]:
# os.environ["ALPHA_VANTAGE_KEY"]="60MP74RUOD4MUHKI"
# key=os.environ["ALPHA_VANTAGE_KEY"]

In [ ]:
# alphav_url="https://www.alphavantage.co/query"
# params={
#     "apikey":key,
#     "function":"TIME_SERIES_DAY",
#     #'interval': '5min',
#     "symbol":None,
#     "datatype":"csv",
#     "outputsize":"compact"
# }

In [ ]:
# nelements=len(sample)
# df=pd.DataFrame()
# ix=0
# #ticker="XOM"
# for (sector,nobs), ticker in tqdm(sample.iteritems(),total=nelements):
#     params["symbol"]=ticker
#     r=requests.get(alphav_url,params=params)
#     ticker_df=pd.read_csv(StringIO(r.text))

#     ticker_df=(ticker_df.set_index("timestamp")[["close","volume"]].rename({"close":f"close_{ticker}",
#                                                                           "volume":f"volume_{ticker}"},axis=1))


#     df=df.join(ticker_df,how="outer")
    
#     ix +=1
#     if ix % 5==0:
#       sleep(61)

In [ ]:
ticker_df = yfinance.download('VNT',period='1Y')[['Close', 'Volume']].rename({"Close":f"close",
                                                                           "Volume":f"volume"},axis=1)
ticker_df['V_change'] = (ticker_df['volume'] / ticker_df['volume'].shift(1)) - 1
ticker_df

[*********************100%***********************]  1 of 1 completed


,close,volume,V_change
Date,,,
2020-09-24,34.000000,743500,NaN
2020-09-28,33.000000,150500,-0.797579
2020-09-29,30.500000,220200,0.463123
2020-09-30,31.000000,21900,-0.900545
2020-10-01,34.000000,43600,0.990868
...,...,...,...
2021-03-15,30.850000,3804400,0.711689
2021-03-16,30.830000,1969700,-0.482257
2021-03-17,31.809999,1777100,-0.097781


In [ ]:
# ix=0
# df=pd.DataFrame()
# df_volume = pd.DataFrame()
# for i in tqdm(range(len(sample))):
#   ticker_df = yfinance.download(sample.Symbol[i],period='1Y')[['Close', 'Volume']].rename({"Close":f"close_{sample.Symbol[i]}",
#                                                                                            'Volume':f'volume_{sample.Symbol[i]}'},axis=1)

#   df=pd.DataFrame(ticker_df[f'close_{sample.Symbol[i]}']).join(df,lsuffix='_caller', rsuffix='_other')
#   df_volume = pd.DataFrame(ticker_df[f'volume_{sample.Symbol[i]}']).join(df_volume,lsuffix='_caller', rsuffix='_other')

#   # ix +=1
#   # if ix % 50==0:
#   #   sleep(15)

In [ ]:
ix=0
df=pd.DataFrame()
df_volume = pd.DataFrame()
for i in tqdm(range(len(sample))):
  ticker_df = yfinance.download(sample.Symbol[i],period='1Y')[['Close', 'Volume']].rename({"Close":f"close_{sample.Symbol[i]}",
                                                                                           'Volume':f'volume_{sample.Symbol[i]}'},axis=1)

  df.insert(i, f'close_{sample.Symbol[i]}',ticker_df[f'close_{sample.Symbol[i]}'])
  df_volume.insert(i, f'volume_{sample.Symbol[i]}',ticker_df[f'volume_{sample.Symbol[i]}'])

  # ix +=1
  # if ix % 50==0:
  #   sleep(15)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

ValueError: ignored

In [ ]:
df_volume

In [ ]:
df

In [ ]:
close_count=df>df.shift(1)
close_count=close_count*1
close_count

In [ ]:
streaks=close_count != 0 
streaks

In [ ]:
streaks.cumsum()

In [ ]:
streaks.cumsum().where(~streaks).ffill().fillna(0).astype(int)

In [ ]:
df_streaks1=streaks.cumsum()-streaks.cumsum().where(~streaks).ffill().fillna(0).astype(int)
df_streaks1

In [ ]:
volume_change = (df_volume / df_volume.shift(1)) - 1
volume_change

In [ ]:
volume_change2 = volume_change.reset_index().melt(id_vars="Date")
volume_change2["variable"]=volume_change2["variable"].apply(lambda v: v.replace("volume_",""))
volume_change2 = volume_change2.rename({"value":"V_change"},axis=1)
volume_change2

In [ ]:
df_streaks2=df_streaks1.reset_index().melt(id_vars="Date")
df_streaks2["variable"]=df_streaks2["variable"].apply(lambda v: v.replace("close_",""))
df_streaks2

In [ ]:
df_streaks3=df_streaks2.merge(sample.reset_index(),left_on="variable",right_on="Symbol")
df_streaks3.insert(7,'V_change',volume_change2['V_change'].tolist())
df_streaks3

In [ ]:
df_streaks3["y"]=(df_streaks3["value"].shift(-1)>0)*1
df_streaks3

In [ ]:
df_streaks4 = df_streaks3[df_streaks3['value'] > 7]
#df_streaks4 = df_streaks3

df_streaks4

In [ ]:
train, test=train_test_split(df_streaks4,test_size=.2,random_state=1234)

xtrain, ytrain = train[['value', 'GICS Sector', 'GICS Sub-Industry', 'V_change']], train["y"]
xtest, ytest = test[['value', 'GICS Sector', 'GICS Sub-Industry', 'V_change']], test["y"]

In [ ]:
xtrain

In [ ]:
# X=df_streaks4[["value","GICS Sector"]]
# y=df_streaks4[["y"]]

In [ ]:
xtrain['GICS Sector'].value_counts()

In [ ]:
ytrain.value_counts()

In [ ]:
categories = sample['GICS Sector'].unique().tolist() + sample['GICS Sub-Industry'].unique().tolist()

In [ ]:
csel=ColumnTransformer([
    ("ohe",OneHotEncoder(sparse=False),["GICS Sector", 'GICS Sub-Industry']),
    ('si', SimpleImputer(), ['V_change']) #,categories=[categories]
],remainder="passthrough")

pipe=Pipeline([
    ("col_sel",csel),
    ("Árbol",DecisionTreeClassifier())
])

In [ ]:
csel.fit_transform(xtrain).shape

In [ ]:
%%time

scores=cross_validate(pipe,xtrain,ytrain,cv=10,scoring="accuracy", return_train_score=True)
scores=scores["test_score"]
accuracy=scores.mean()

print(f"Promedio Cross-Validation (Pre-optimización):{accuracy:>15,.2f}")

In [ ]:
# depths = np.linspace(2,10,9)
# min_samples_split = np.array([2,3,4,5,6,7,8,9,10])
# min_samples_leaf = np.array([1,2,3,4,5])
# criterion = np.array(['gini','entropy'])

In [ ]:
# %%time

# pipe.fit(xtrain,ytrain)

# grid={
#     "Árbol__max_depth":depths,
#     "Árbol__min_samples_split":min_samples_split,
#     "Árbol__min_samples_leaf":min_samples_leaf,
#     "Árbol__criterion":criterion
# }

# gsearch=GridSearchCV(pipe,grid, scoring="accuracy", return_train_score=True, cv=10)
# gsearch.fit(xtrain,ytrain)
# cvres=gsearch.cv_results_

In [ ]:
# cv=cross_validate(gsearch.best_estimator_,xtrain,ytrain,cv=10,scoring=["accuracy"])
# mean_acc = cv['test_accuracy'].mean()
# print(f"Media Cross-Validation (Post-optimización):{mean_acc:>15,.2f}")

In [ ]:
#gsearch.best_estimator_['Árbol']

In [ ]:
#Versión Árbol de Decisión

pipe=Pipeline([
    ("col_sel",csel),
    ("Árbol",DecisionTreeClassifier(criterion='gini', max_depth=10.0, min_samples_leaf=1, min_samples_split=2))
])

pipe.fit(xtrain,ytrain)
ypred=pipe.predict(xtest)

accuracy = accuracy_score(ytest,ypred)
print(f"Accuracy del modelo Árbol de Decisión: {accuracy:,.3f}")

In [ ]:
categories + ["ndays"] 

In [ ]:
# plt.figure(dpi=300,figsize=(28,12))
# plot_tree(pipe.named_steps["Árbol"],
#          feature_names=categories + ["ndays"],
#          class_names=["down","up"],
#          fontsize=7,
#          rounded=True,
#          filled=True);

In [ ]:
#VERSION RANDOM FOREST

%%time

pipe=Pipeline([
    ("col_sel",csel),
    ("Árbol",RandomForestClassifier(n_estimators=500,random_state=314,n_jobs=-1,oob_score=True,max_samples=100))
])
#criterion='entropy', min_samples_leaf=1, min_samples_split=6, max_depth=9.0,

pipe.fit(xtrain,ytrain)
ypred=pipe.predict(xtest)

accuracy = accuracy_score(ytest,ypred)
print(f"Accuracy del modelo Randome Forest: {accuracy:,.3f}")

In [ ]:
#VERSION REGRESIÓN LOGISTICA

pipe=Pipeline([        
    ("col_sel",csel),
    ("logística",LogisticRegression(max_iter=2000))  
])

pipe.fit(xtrain,ytrain)
ypred=pipe.predict(xtest)

accuracy = accuracy_score(ytest,ypred)
print(f"Accuracy del modelo Reg Logística: {accuracy:,.3f}")

In [ ]:
#VERSION SVC

# pipe=Pipeline([        
#     ("col_sel",csel),
#     ("SVC_linear",SVC(kernel='linear', C=.1, gamma=0.01, probability=True))  
# ])

# pipe.fit(xtrain,ytrain)
# ypred=pipe.predict(xtest)

# accuracy = accuracy_score(ytest,ypred)
# print(f"Accuracy del modelo SVC Linear: {accuracy:,.2f}")

In [ ]:
# pipe=Pipeline([        
#     ("col_sel",csel),
#     ("SVC_poli",SVC(kernel='poly', C=.1, gamma=0.1, probability=True))  
# ])

# pipe.fit(xtrain,ytrain)
# ypred=pipe.predict(xtest)

# accuracy = accuracy_score(ytest,ypred)
# print(f"Accuracy del modelo SVC Poly: {accuracy:,.2f}")

In [ ]:
# pipe=Pipeline([        
#     ("col_sel",csel),
#     ("SVC_rbf",SVC(kernel='rbf', C=1, gamma=0.1, probability=True))  
# ])

# pipe.fit(xtrain,ytrain)
# ypred=pipe.predict(xtest)

# accuracy = accuracy_score(ytest,ypred)
# print(f"Accuracy del modelo Reg Logística: {accuracy:,.2f}")

In [ ]:
# pipe=Pipeline([        
#     ("col_sel",csel),
#     ("SVC_sigmoid",SVC(kernel='sigmoid', C=1, gamma=0.01))  
# ])

# pipe.fit(xtrain,ytrain)
# ypred=pipe.predict(xtest)

# accuracy = accuracy_score(ytest,ypred)
# print(f"Accuracy del modelo Reg Logística: {accuracy:,.2f}")